# 0. Importando bibliotecas

Para este código você precisara apenas de Pandas e Numpy

In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

# 1. Importando dados

## 1.1. Formato dos dados

Para executar esse código, você precisará organiza-los da maneira correta, são necessárias 2 tabelas.

### 1.1.1. Tabela com dados de cada cluster

A tabela com os dados de cada cluster necessita das seguintes colunas:

- **CLUSTER**: Contem o identificador único do Cluster (pode conter qualquer formato, desde que seja único para cada cluster);
- **ORIGEM**: Nome da coluna que contem o atributo (que é armazenado na próxima coluna);
- **ATRIBUTO**: Valor do atributo que fazia parte de uma coluna ORIGEM (recomenda-se utilizar faixas ou valores binários);
- **COUNT**: Contagem de linhas que possuem aquele ATRIBUTO naquela ORIGEM naquele CLUSTER;

Para explicar melhor a relação entre ORIGEM → PRODUTO, vamos considerar o seguinte exemplo:

|CLASSE SOCIAL|
|---|
|A|
|B|
|C|

Na tabela acima vemos que CLASSE SOCIAL seria a ORIGEM, e cada valor referente a CLASSE SOCIAL é um ATRIBUTO, ficando da seguinte forma:

|ORIGEM|ATRIBUTO|
|---|---|
|CLASSE SOCIAL|A|
|CLASSE SOCIAL|B|
|CLASSE SOCIAL|C|

### 1.1.2. Tabela com dados de para

A tabela de para, serve para definir os significados de cada ORIGEM e ATRIBUTO, além de definir se aquela combinação ORIGEM / ATRIBUTO deverá ser destacada caso seja uma informação relevante para o algoritmo.

A tabela deverá seguir o seguinte padrão de colunas:

- **ORIGEM**: Nome da coluna que contem o atributo (que é armazenado na próxima coluna);
- **ATRIBUTO**: Valor do atributo que fazia parte de uma coluna ORIGEM (recomenda-se utilizar faixas ou valores binários);
- **PROMPT_ORIGEM**: Breve descrição sobre o que é a coluna ORIGEM daquela linha ();
- **PROMPT_ATRIBUTO**: Breve descrição sobre o que é o ATRIBUTO daquela linha;
- **CONSIDERAR**: True ou False, definem se a combinação ORIGEM / ATRIBUTO deverá ser destacada seja relevante.

## 1.2. Join das tabelas de dados dos clusters e a tabela de de para

In [2]:
# Dados sobre os clusters
dados = pd.read_excel('../data/processed/profile_clustered.xlsx', sheet_name='DADOS')

# Dados sobre variáveis
depara = pd.read_excel('../data/processed/profile_clustered.xlsx', sheet_name='DE_PARA')

# Quais atributos deve-se considerar ou não
dados = dados.merge(depara[['ORIGEM','ATRIBUTO','PROMPT_ORIGEM','PROMPT_ATRIBUTO','CONSIDERAR']], on=['ORIGEM','ATRIBUTO'], how='left')
display(dados.head(10))

del(depara)

,CLUSTER,ORIGEM,ATRIBUTO,COUNT,PROMPT_ORIGEM,PROMPT_ATRIBUTO,CONSIDERAR
0,0,credit_card_limit,00_Sem_dados,1151,limite do cartão de crédito,dado não disponivel,False
1,0,credit_card_limit,01_Abaixo_de_49000,625,limite do cartão de crédito,abaixo de 49k,True
2,0,credit_card_limit,02_Entre_49000_e_63000,552,limite do cartão de crédito,49k a 63k,True
3,0,credit_card_limit,03_Entre_63000_e_79000,424,limite do cartão de crédito,63k a 79k,True
4,0,credit_card_limit,04_Acima_de_79000,290,limite do cartão de crédito,acima de 79k,True
5,1,credit_card_limit,00_Sem_dados,27,limite do cartão de crédito,dado não disponivel,False
6,1,credit_card_limit,01_Abaixo_de_49000,195,limite do cartão de crédito,abaixo de 49k,True
7,1,credit_card_limit,02_Entre_49000_e_63000,353,limite do cartão de crédito,49k a 63k,True
8,1,credit_card_limit,03_Entre_63000_e_79000,398,limite do cartão de crédito,63k a 79k,True
9,1,credit_card_limit,04_Acima_de_79000,432,limite do cartão de crédito,acima de 79k,True


# 2. Calculando percentuais de representação dentro dos clusters

Aqui, será realizado o calculo de percentual de quanto que aquele atributo representa dentro do cluster.

A representação dentro do cluster é utilizado junto com o LIFT para verificar se a informação deverá ser destacada.

In [3]:
try:
    dados.drop('PERCENTUAL', inplace=True, axis=1)
    dados.drop('LIFT', inplace=True, axis=1)
    dados.drop('LIFT_CLASS', inplace=True, axis=1)
    dados.drop('REP_CLASS', inplace=True, axis=1)
except: pass

# Todas as possíveis Origens
all_origin = dados['ORIGEM'].unique()

# Dataframe contendo o percentual por atributo sem filtrar clusters
percentual_per_atr = pd.DataFrame(columns=['ORIGEM','ATRIBUTO','PERCENUTAL'])

# Calculando % de representatividade em cada cluster para cada ORIGEM
all_percentages = pd.DataFrame(columns=['CLUSTER','ORIGEM','ATRIBUTO','PERCENUTAL'])
for origin in all_origin:
    # Filtrando para obter apenas os dados da ORIGEM
    temp = dados[dados['ORIGEM'] == origin]

    # Coletando todos os clusters
    all_clusters = temp['CLUSTER'].unique()

    # Passando por cada cluster
    for cl in all_clusters:
        temp_cl = temp[temp['CLUSTER'] == cl]

        # Calculo do percentual
        ttl_count = temp_cl['COUNT'].sum()
        # Calculo do percentual por cluster
        temp_cl['PERCENTUAL'] = temp_cl['COUNT'] * 100 / ttl_count

        # Concatenando
        all_percentages = pd.concat([all_percentages, temp_cl[['CLUSTER','ORIGEM','ATRIBUTO','PERCENTUAL']]])

    # Percentual sem filtrar clusters
    # Agrupando para remover os clusters
    temp_g = temp.groupby(['ORIGEM','ATRIBUTO'], as_index=False).agg({'COUNT':'sum'})
    # Contagem total de dados sem atributos
    ttl_count = temp_g['COUNT'].sum()
    # Calculo do percentual geral
    temp_g['PERCENTUAL'] = temp_g['COUNT'] * 100 / ttl_count
    # Adicionando a tabela de percentuais sem cluster
    percentual_per_atr = pd.concat([percentual_per_atr, temp_g])

# Realizando merge
dados = dados.merge(all_percentages[['CLUSTER','ORIGEM','ATRIBUTO','PERCENTUAL']], on=['CLUSTER','ORIGEM','ATRIBUTO'], how='left')

# Exibindo coisas
display(dados.head(10))
display(percentual_per_atr.head(10))

,CLUSTER,ORIGEM,ATRIBUTO,COUNT,PROMPT_ORIGEM,PROMPT_ATRIBUTO,CONSIDERAR,PERCENTUAL
0,0,credit_card_limit,00_Sem_dados,1151,limite do cartão de crédito,dado não disponivel,False,37.836949
1,0,credit_card_limit,01_Abaixo_de_49000,625,limite do cartão de crédito,abaixo de 49k,True,20.545694
2,0,credit_card_limit,02_Entre_49000_e_63000,552,limite do cartão de crédito,49k a 63k,True,18.145957
3,0,credit_card_limit,03_Entre_63000_e_79000,424,limite do cartão de crédito,63k a 79k,True,13.938199
4,0,credit_card_limit,04_Acima_de_79000,290,limite do cartão de crédito,acima de 79k,True,9.533202
5,1,credit_card_limit,00_Sem_dados,27,limite do cartão de crédito,dado não disponivel,False,1.921708
6,1,credit_card_limit,01_Abaixo_de_49000,195,limite do cartão de crédito,abaixo de 49k,True,13.879004
7,1,credit_card_limit,02_Entre_49000_e_63000,353,limite do cartão de crédito,49k a 63k,True,25.124555
8,1,credit_card_limit,03_Entre_63000_e_79000,398,limite do cartão de crédito,63k a 79k,True,28.327402
9,1,credit_card_limit,04_Acima_de_79000,432,limite do cartão de crédito,acima de 79k,True,30.747331


,ORIGEM,ATRIBUTO,PERCENUTAL,COUNT,PERCENTUAL
0,credit_card_limit,00_Sem_dados,NaN,2175.0,12.794118
1,credit_card_limit,01_Abaixo_de_49000,NaN,3781.0,22.241176
2,credit_card_limit,02_Entre_49000_e_63000,NaN,3605.0,21.205882
3,credit_card_limit,03_Entre_63000_e_79000,NaN,3722.0,21.894118
4,credit_card_limit,04_Acima_de_79000,NaN,3717.0,21.864706
0,age,00_Sem_dados,NaN,2175.0,12.794118
1,age,02_Entre_18_e_29,NaN,1574.0,9.258824
2,age,03_Entre_30_e_39,NaN,1526.0,8.976471
3,age,04_Entre_40_e_49,NaN,2309.0,13.582353
4,age,05_Entre_50_e_59,NaN,3541.0,20.829412


# 3. Calculando o Lift de cada ORIGEM/ATRIBUTO

Nesta etapa, o lift será calculado. O lift define se aquele atributo está com uma representação muito mais alta ou baixa que a média dele mesmo entre os outros clusters.

Isso ajuda a entender se aquele atributo está fora do padrão do restante da população.

In [4]:
try:
    dados.drop('LIFT', inplace=True, axis=1)
    dados.drop('LIFT_CLASS', inplace=True, axis=1)
    dados.drop('REP_CLASS', inplace=True, axis=1)
except: pass

# Calculando o lift de cada atributo em cada cluster
all_lift = pd.DataFrame(columns=['CLUSTER','ORIGEM','ATRIBUTO','PERCENUTAL','LIFT'])

# Coletando todas as origens
all_origin = dados['ORIGEM'].unique()

# Dataframe para salvar o lift de cada atributo em cada origem e cluster
all_lift = pd.DataFrame(columns=['CLUSTER','ORIGEM','ATRIBUTO','PERCENUTAL','LIFT'])
for origin in all_origin:
    # Filtrando para obter apenas os dados da ORIGEM
    temp = dados[dados['ORIGEM'] == origin]
    # Todos os atributos
    all_atrs = temp['ATRIBUTO'].unique()
    for atr in all_atrs:
        # Filtrando atributo
        temp_atr = temp[temp['ATRIBUTO'] == atr]
        # Calculando lift
        temp_atr['LIFT'] = temp_atr['PERCENTUAL'] / percentual_per_atr[(percentual_per_atr['ORIGEM'] == origin) & (percentual_per_atr['ATRIBUTO'] == atr)]['PERCENTUAL'].values[0]
        # Concatenando calculo
        all_lift = pd.concat([all_lift, temp_atr])

# Realizando merge
dados = dados.merge(all_lift[['CLUSTER','ORIGEM','ATRIBUTO','LIFT']], on=['CLUSTER','ORIGEM','ATRIBUTO'], how='left')

# Exibindo coisas
display(dados.head(10))

C:\Users\dpgab\AppData\Local\Temp\ipykernel_20048\1367411980.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_lift = pd.concat([all_lift, temp_atr])


,CLUSTER,ORIGEM,ATRIBUTO,COUNT,PROMPT_ORIGEM,PROMPT_ATRIBUTO,CONSIDERAR,PERCENTUAL,LIFT
0,0,credit_card_limit,00_Sem_dados,1151,limite do cartão de crédito,dado não disponivel,False,37.836949,2.957371
1,0,credit_card_limit,01_Abaixo_de_49000,625,limite do cartão de crédito,abaixo de 49k,True,20.545694,0.923768
2,0,credit_card_limit,02_Entre_49000_e_63000,552,limite do cartão de crédito,49k a 63k,True,18.145957,0.855704
3,0,credit_card_limit,03_Entre_63000_e_79000,424,limite do cartão de crédito,63k a 79k,True,13.938199,0.636618
4,0,credit_card_limit,04_Acima_de_79000,290,limite do cartão de crédito,acima de 79k,True,9.533202,0.436009
5,1,credit_card_limit,00_Sem_dados,27,limite do cartão de crédito,dado não disponivel,False,1.921708,0.150202
6,1,credit_card_limit,01_Abaixo_de_49000,195,limite do cartão de crédito,abaixo de 49k,True,13.879004,0.624023
7,1,credit_card_limit,02_Entre_49000_e_63000,353,limite do cartão de crédito,49k a 63k,True,25.124555,1.184792
8,1,credit_card_limit,03_Entre_63000_e_79000,398,limite do cartão de crédito,63k a 79k,True,28.327402,1.293836
9,1,credit_card_limit,04_Acima_de_79000,432,limite do cartão de crédito,acima de 79k,True,30.747331,1.406254


# 4. Classificando o Lift

## 4.1. Introdução

A classificação do lift serve para definir quais atributos no cluster possuem os maiores e menores LIFT.

A classificação depende dos percentis do lift dentro de cada cluster.

## 4.2. Configurando

Para configurar a classificação do LIFT, customize o código abaixo, pois ele permite que você defina seus próprios thresholds de LIFT e até customizar os nomes.

Caso os nomes sejam alterados, lembre-se de modificar a lista no item 6.2.

In [5]:
__LIFT_THRESHOLDS__ = {
    'ALTISSIMO':  0.90,
    'ALTO':       0.75,
    'MEDIO':      0.50,
    'BAIXO':      0.25,
    # !!! BAIXISSIMO será classificado como os valores
    # entre 0 e o ultimo thershold definido !!!
}
__LOWEST_LIFT_CLASS__ = 'BAIXISSIMO'

## 4.3. Execução

In [6]:
try:
    dados.drop('LIFT_CLASS', inplace=True, axis=1)
    dados.drop('REP_CLASS', inplace=True, axis=1)
except: pass

# DataFrame com todos os lift classificados
classified_lift = pd.DataFrame(columns=['CLUSTER','ORIGEM','ATRIBUTO','PERCENUTAL','LIFT','LIFT_CLASS'])

# Listando clusters unicos
all_clusters = dados['CLUSTER'].unique()

# Passando por cada cluster para classificar o Lift de cada atributo
for cl in all_clusters:
    temp = dados[dados['CLUSTER'] == cl]
    # Variável para salvar os valores de cada threshold
    lift_limits = dict()
    # Coletando o percentil de cada threshold (revertendo ordem para a
    # próxima estapa começar com BAIXISSIMO)
    for label, threshold in reversed(__LIFT_THRESHOLDS__.items()):
        # Coletando o percentil desejado
        lift_limits[label] = np.quantile(temp['LIFT'], threshold)

    # Variavel que terá o ultimo limite
    last_threshold = 0
    # Classificando o Lift
    for label, threshold in lift_limits.items():
        temp_filtred = temp[(temp['LIFT'] >= last_threshold) & (temp['LIFT'] < threshold)]
        # Caso essa seja a primeira classificação
        if last_threshold == 0: temp_filtred['LIFT_CLASS'] = __LOWEST_LIFT_CLASS__
        # Caso já tenha passado pela primeira classificação
        else: temp_filtred['LIFT_CLASS'] = last_label
        # Redefinindo o ultimo threshold e classificação
        last_threshold = threshold
        last_label = label
        # Salvando as alterações num dataframe
        classified_lift = pd.concat([classified_lift, temp_filtred])
    # Classificando os que estão acima do ultimo label (provavelmente ALTISSIMO)
    temp_filtred = temp[temp['LIFT'] >= last_threshold]
    temp_filtred['LIFT_CLASS'] = last_label
    # Salvando as alterações num dataframe
    classified_lift = pd.concat([classified_lift, temp_filtred])

# Realizando merge
dados = dados.merge(classified_lift[['CLUSTER','ORIGEM','ATRIBUTO','LIFT_CLASS']], on=['CLUSTER','ORIGEM','ATRIBUTO'], how='left')

# Exibindo coisas
display(dados.head(10))

C:\Users\dpgab\AppData\Local\Temp\ipykernel_20048\449406733.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  classified_lift = pd.concat([classified_lift, temp_filtred])


,CLUSTER,ORIGEM,ATRIBUTO,COUNT,PROMPT_ORIGEM,PROMPT_ATRIBUTO,CONSIDERAR,PERCENTUAL,LIFT,LIFT_CLASS
0,0,credit_card_limit,00_Sem_dados,1151,limite do cartão de crédito,dado não disponivel,False,37.836949,2.957371,ALTO
1,0,credit_card_limit,01_Abaixo_de_49000,625,limite do cartão de crédito,abaixo de 49k,True,20.545694,0.923768,BAIXO
2,0,credit_card_limit,02_Entre_49000_e_63000,552,limite do cartão de crédito,49k a 63k,True,18.145957,0.855704,BAIXO
3,0,credit_card_limit,03_Entre_63000_e_79000,424,limite do cartão de crédito,63k a 79k,True,13.938199,0.636618,BAIXISSIMO
4,0,credit_card_limit,04_Acima_de_79000,290,limite do cartão de crédito,acima de 79k,True,9.533202,0.436009,BAIXISSIMO
5,1,credit_card_limit,00_Sem_dados,27,limite do cartão de crédito,dado não disponivel,False,1.921708,0.150202,BAIXISSIMO
6,1,credit_card_limit,01_Abaixo_de_49000,195,limite do cartão de crédito,abaixo de 49k,True,13.879004,0.624023,BAIXO
7,1,credit_card_limit,02_Entre_49000_e_63000,353,limite do cartão de crédito,49k a 63k,True,25.124555,1.184792,MEDIO
8,1,credit_card_limit,03_Entre_63000_e_79000,398,limite do cartão de crédito,63k a 79k,True,28.327402,1.293836,MEDIO
9,1,credit_card_limit,04_Acima_de_79000,432,limite do cartão de crédito,acima de 79k,True,30.747331,1.406254,MEDIO


# 5. Classificando a Represntação

## 5.1. Introdução

A classificação de representação ajuda a entender a escala daquela representação. 

## 5.2. Configurando

Aqui podemos configurar os thresholds de cada escala de representação, alterando até os nomes.

Caso os nomes sejam alterados, lembre-se de modificar a lista no item 6.2.

In [7]:
__REP_THRESHOLDS__ = {
    'ALTISSIMO':  50,
    'ALTO':       40,
    'MEDIO':      30,
    'BAIXO':      5,
    # !!! BAIXISSIMO será classificado como os valores
    # entre 0 e o ultimo thershold definido !!!
}
__LOWEST_REP_CLASS__ = 'BAIXISSIMO'

## 5.3. Execução

In [8]:
try: dados.drop('REP_CLASS', inplace=True, axis=1)
except: pass

last_threshold = 0
for label, threshold in reversed(__REP_THRESHOLDS__.items()):
    # Salvando as alterações num dataframe
    if last_threshold == 0: dados.loc[dados[(dados['PERCENTUAL'] >= last_threshold) & (dados['PERCENTUAL'] < threshold)].index, 'REP_CLASS'] = __LOWEST_REP_CLASS__
    else: dados.loc[dados[(dados['PERCENTUAL'] >= last_threshold) & (dados['PERCENTUAL'] < threshold)].index, 'REP_CLASS'] = last_label

    # Redefinindo o ultimo threshold e classificação
    last_threshold = threshold
    last_label = label

# Classificando ALTISSIMO
dados.loc[dados[(dados['PERCENTUAL'] >= last_threshold)].index, 'REP_CLASS'] = last_label

# Exibindo coisas
display(dados.head(10))

,CLUSTER,ORIGEM,ATRIBUTO,COUNT,PROMPT_ORIGEM,PROMPT_ATRIBUTO,CONSIDERAR,PERCENTUAL,LIFT,LIFT_CLASS,REP_CLASS
0,0,credit_card_limit,00_Sem_dados,1151,limite do cartão de crédito,dado não disponivel,False,37.836949,2.957371,ALTO,MEDIO
1,0,credit_card_limit,01_Abaixo_de_49000,625,limite do cartão de crédito,abaixo de 49k,True,20.545694,0.923768,BAIXO,BAIXO
2,0,credit_card_limit,02_Entre_49000_e_63000,552,limite do cartão de crédito,49k a 63k,True,18.145957,0.855704,BAIXO,BAIXO
3,0,credit_card_limit,03_Entre_63000_e_79000,424,limite do cartão de crédito,63k a 79k,True,13.938199,0.636618,BAIXISSIMO,BAIXO
4,0,credit_card_limit,04_Acima_de_79000,290,limite do cartão de crédito,acima de 79k,True,9.533202,0.436009,BAIXISSIMO,BAIXO
5,1,credit_card_limit,00_Sem_dados,27,limite do cartão de crédito,dado não disponivel,False,1.921708,0.150202,BAIXISSIMO,BAIXISSIMO
6,1,credit_card_limit,01_Abaixo_de_49000,195,limite do cartão de crédito,abaixo de 49k,True,13.879004,0.624023,BAIXO,BAIXO
7,1,credit_card_limit,02_Entre_49000_e_63000,353,limite do cartão de crédito,49k a 63k,True,25.124555,1.184792,MEDIO,BAIXO
8,1,credit_card_limit,03_Entre_63000_e_79000,398,limite do cartão de crédito,63k a 79k,True,28.327402,1.293836,MEDIO,BAIXO
9,1,credit_card_limit,04_Acima_de_79000,432,limite do cartão de crédito,acima de 79k,True,30.747331,1.406254,MEDIO,MEDIO


# 6. Gerando resumo de todos os clusters

## 6.1. Introdução

Aqui, um conjunto de condições serão aplicadas em cada linha da tabela para montar o texto de destaques de informações de cada cluster.

## 6.2. Configurando

Aqui, modifique a lista de condições para se destacar as informações mais relevantes.

Na parte de baixo, configure o texto introdutório a sequencia de destaques de cada cluster.

In [9]:
# Definição de vinculo entre REPRESENTAÇÃO e LIFT
__REP_X_LIFT__ = {
    'ALTISSIMO':  ['BAIXISSIMO', 'BAIXO', 'MEDIO', 'ALTO', 'ALTISSIMO'],
    'ALTO':       ['BAIXO', 'MEDIO', 'ALTO', 'ALTISSIMO'],
    'MEDIO':      ['MEDIO', 'ALTO', 'ALTISSIMO'],
    'BAIXO':      ['ALTO', 'ALTISSIMO'],
    'BAIXISSIMO': ['ALTISSIMO'],
}

# Texto inicial
__STARTING_TEXT__ = '''
Descreva as principais carateristicas que difernciam cada um dos clusters de clientes:
'''

## 6.3. Execução

In [10]:
# Listando todos os clusters
all_clusters = dados['CLUSTER'].unique()

# String que conterá todas as informações destacadas sobre os clusters e que
# será salvo como um txt que você pode copiar e colar no Copilot ou qualquer GenAI
highlight_text = ''

# Passando por todos os clusters
for cl in all_clusters:
    # Começando texto para o cluster
    highlight_text += f'\n\n\nDestaques do cluster {cl}\n{__STARTING_TEXT__}'

    # Listando todas as origens
    all_origins = dados['ORIGEM'].unique()

    # Passando por cada origem para destacar o que é mais relevante
    for origin in all_origins:
        # Filtrando base para obter apenas dados da origem vinculados ao cluster
        temp = dados[(dados['CLUSTER'] == cl) & (dados['ORIGEM'] == origin)]

        # Filtrando destacaveis
        valid = pd.concat([temp[(temp['LIFT_CLASS'] == lift_threshold) & (temp['REP_CLASS'].isin(rep_thresholds)) & (temp['CONSIDERAR'] == True)] for lift_threshold, rep_thresholds in __REP_X_LIFT__.items()]).drop_duplicates()

        # Passando por cada linha valida
        if len(valid) > 0:
            # Criando começo do destaque
            highlight_text += f" - Na caracteristica {temp['PROMPT_ORIGEM'].values[0]} destaca-se "
            # Lista para salvar informações destacadas
            info_list = []
            # Passando por cada informação
            for _, row in valid.iterrows():
                # Salvando textos na lista de destaques
                info_list.append(f"{row['PROMPT_ATRIBUTO']} (representa {round(row['PERCENTUAL'])}% do cluster e possui lift {row['LIFT_CLASS']})")
            # Adicionar os textos
            highlight_text += ', '.join(info_list) + ';\n'

# 7. Salvando em um txt

In [11]:
with open('../data/processed/clusters_description.txt', mode='w', encoding='utf-8') as f:
    f.write(highlight_text)

Agora envie o arquivo para uma LLM e aguarde para ver o resultado após digitar o seguinte prompt:

> Estou realizando uma analise de perfil de oito clusters de clientes de uma base de um aplicativo de compra de alimentos entregues por delivery.
>
> Quero que para cada cluster você gere um nome descritivo e um a dois parágrafos onde você aponte as características mais relevantes. Quando um cluster for similar ao outro, diga que é similar e destaque o que difere um do outro.